In [3]:
%matplotlib inline
import numpy as np
import xarray as xr
import cubedsphere as cs

In [4]:
Vres_list = [20,40,80,160]
Hres_list = [48,96,192,384]
eps = 1e-10

for i,Hres in enumerate(Hres_list):
    for j,Vres in enumerate(Vres_list):
        print('loading',Hres,Vres,end=' ')
        datadir = ("../../output/C{0}/C{0}L{1}/".format(Hres,Vres))
        ds = cs.open_FV3data(datadir,"tracer_daily")
        dr = ds['plume01'].load()
        
        dr.values[dr.values<eps] = eps # correct negative values
        
        print('calculating...',end=' ')
        # higher-Vres layers are thiner
        dr_entro = (-dr*np.log(dr)*dr['area']/1e9/Vres).sum(dim=['tile','pfull','y','x'])
        
        print('writting...',end='\n')
        dr_entro.to_netcdf('plume01_entropy_C{0}L{1}'.format(Hres,Vres))

loading 48 20 calculating... writting...
loading 48 40 calculating... writting...
loading 48 80 calculating... writting...
loading 48 160 calculating... writting...
loading 96 20 calculating... writting...
loading 96 40 calculating... writting...
loading 96 80 calculating... writting...
loading 96 160 calculating... writting...
loading 192 20 calculating... writting...
loading 192 40 calculating... writting...
loading 192 80 calculating... writting...
loading 192 160 calculating... writting...
loading 384 20 calculating... writting...
loading 384 40 calculating... writting...
loading 384 80 calculating... writting...
loading 384 160 calculating... writting...


## Merge all files

In [5]:
ds_HsVs_arr = [] # list of timeseries at different horizontal and vertical resolutions

for Hres in Hres_list:
    ds_Vs_arr = [] # list of timeseries at vertical different resolutions
    for Vres in Vres_list:
        
        dr_temp =  xr.open_dataarray('plume01_entropy_C{0}L{1}'.format(Hres,Vres),decode_times=False)
        ds_Vs_arr.append( dr_temp.copy()) 
            
        ds_Vs = xr.concat(ds_Vs_arr, dim='Vres')
        ds_Vs["Vres"]=np.array(Vres_list)
        
    ds_HsVs_arr.append( ds_Vs )

ds_HsVs = xr.concat(ds_HsVs_arr, dim='Hres')
ds_HsVs["Hres"]=np.array(Hres_list)

In [6]:
ds_HsVs.to_netcdf('plume01_entropy_all.nc')